# Brains4Buildings interactive inverse grey-box analysis pipeline

This Jupyter Labs notebook can be used to interactively test the Brains4Buildings inverse grey-box analysis pipeline.
Don't forget to install the requirements listed in [requirements.txt](../requirements.txt) first!



## Setting the stage

First several imports and variables need to be defined


### Imports and generic settings

In [ ]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

from tqdm.notebook import tqdm
from gekko import GEKKO

import sys
sys.path.append('../data/')
sys.path.append('../view/')
sys.path.append('../analysis/')

from preprocessor import Preprocessor
from inversegreyboxmodel import Learner
from plotter import Plot


%load_ext autoreload
%matplotlib inline
%matplotlib widget

### Load Measured Data from parquet file

In [ ]:
%%time
# Prerequisite: for this example to work, you need to have the b4b_raw_properties.parquet, located in the ../data/ folder.
# One way to get this is to run B4BExtractionBackup.ipynb first, but then you have to run this code on the energietransitiewindesheim.nl server

df_prop = pd.read_parquet('../data/b4b_raw_properties.parquet', engine='pyarrow')

#sorting the DataFrame index is needed to get good performance on certain filters
#this guarding code to check whether DataFramews are properly sorted
if not df_prop.index.is_monotonic_increasing:
    print('df needed index sorting')
    df_prop = df_prop.sort_index()  

In [ ]:
df_prop.index.unique(level='id').values

In [ ]:
df_prop.index.unique(level='source').values

In [ ]:
df_prop

In [ ]:
df_prop.info()

## Preprocessing

### Example: processing temp_in__degC

In [ ]:
#if this plot does not show up at initial run, run the cell again (something fishy with interactive plotting of DataFrame.plot.hist())
%matplotlib widget
df_prop.temp_in__degC.plot.hist(bins=50, alpha=0.5)

### Example: using the min-max filter
Filtering out extreme, highly unlikely temperature measurements

In [ ]:
Preprocessor.filter_min_max(df_prop, 'temp_in__degC', min=5, max=40)

In [ ]:
df_prop.info()

In [ ]:
df_prop.describe()

In [ ]:
#if this plot does not show up at initial run, run the cell again (something fishy with interactive plotting of DataFrame.plot.hist())
%matplotlib widget
df_prop.temp_in__degC.plot.hist(bins=50, alpha=0.5)

### Example: using the static outlier filter, per id
Filtering out extreme temperatures based on mean and standard deviation per room

In [ ]:
Preprocessor.filter_static_outliers(df_prop, 'temp_in__degC', n_sigma=3.0, per_id=True)

In [ ]:
#if this plot does not show up at initial run, run the cell again (something fishy with interactive plotting of DataFrame.plot.hist())
%matplotlib widget
df_prop.temp_in__degC.plot.hist(bins=50, alpha=0.5)

In [ ]:
df_prop.info()

In [ ]:
df_prop.describe()

### Interpolating all properties


In [ ]:
%%time 
%autoreload 2

property_types = {
    'temp_in__degC' : 'float32',
    'co2__ppm' : 'float32',
    'rel_humidity__0' : 'float32',
    'valve_frac__0' : 'float32',
    'door_open__bool': 'Int8',
    'window_open__bool': 'Int8',
    'occupancy__bool': 'Int8',
    'occupancy__p' : 'Int8'
}

df_interpolated = Preprocessor.interpolate_time(df_prop,
                                        property_dict = property_types,
                                        upsample__min = 5,
                                        interpolate__min = 15,
                                        limit__min = 90,
                                        inplace=False
                                       )

In [ ]:
df_interpolated

In [ ]:
df_interpolated.info()

### Converting raw properties dataframe to preprocessed dataframe


In [ ]:
df_prep = Preprocessor.unstack_prop(df_interpolated)

In [ ]:
df_prep

In [ ]:
df_prep.info()

## Learn parameters using inverse grey-box analysis

Most of the heavy lifting is done by the `learn_room_parameters()` function, which again uses the [GEKKO Python](https://machinelearning.byu.edu/) dynamic optimization toolkit.

In [ ]:
%%time 
%autoreload 2

# learn the model parameters for only a subset of the room ids and write results to a dataframe

df_results, df_prep = Learner.learn_room_parameters(df_prep.loc[[999169,917810]],
                                                    col_co2__ppm = 'bms_co2__ppm', 
                                                    col_occupancy__p = 'bms_occupancy__bool', 
                                                    col_valve_frac__0 = 'bms_valve_frac__0',
                                                    learn_infilt__m2 = False,
                                                    learn_valve_frac__0 = True,
                                                    learn_occupancy__p = False,
                                                    ev_type=2)

### Result Visualization

In [ ]:
df_results

In [ ]:
df_prep

In [ ]:
%autoreload 2
units_to_mathtext = property_types = {
    'degC' : r'$°C$',
    'ppm' : r'$ppm$',
    '0' : r'$[-]$',
    'bool': r'$0 = False; 1 = True$',
    'p' : r'$persons$'
}

In [ ]:
#Plot all properties from all sources for all ids
Plot.dataframe_preprocessed_plot(df_prep, units_to_mathtext)